<h1 align='center'>Incorporando um modelo de aprendizado de máquina em um aplicativo da Web</h1>

<p align='center'><img src=https://afsystems.com.br/wp-content/uploads/2019/03/20190327_voce_sabe_o_que_e_aplicavitos_web_.jpg width='500'></p>

As técnicas de aprendizado de máquina não se limitam a aplicativos e análises *offline*, e se tornaram o mecanismo preditivo de vários serviços da *web*. Por exemplo, aplicativos populares e úteis de modelos de aprendizado de máquina em aplicativos da *Web* incluem detecção de *spam* em formulários de envio, mecanismos de pesquisa, sistemas de recomendação para mídia ou portais de compras e muito mais.

### Serializando estimadores scikit-learn ajustados

Treinar um modelo de aprendizado de máquina pode ser **computacionalmente caro**. Certamente, não queremos treinar novamente nosso modelo toda vez que fechamos nosso interpretador *Python* e queremos fazer uma nova previsão ou recarregar nosso aplicativo da *web*.

Uma opção para persistência de modelo é o módulo *pickle* embutido do *Python* (https://docs.python.org/3.7/library/pickle.html), que nos permite serializar e desserializar estruturas de objeto *Python* para compactar em *bytecode* para que possamos salvar nosso classificador em seu estado atual e recarregá-lo se quisermos classificar novos exemplos não rotulados, sem precisar que o modelo aprenda com os dados de treinamento novamente.

Antes de executar o código a seguir, certifique-se de ter treinado um modelo de regressão logística *out-of-core* e de tê-lo pronto em sua sessão atual do *Python*:

In [8]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

###### Imorimindo os Mini Batchs
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


In [9]:
# Treinando a Acurácia do modelo
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

clf = clf.partial_fit(X_test, y_test)

Accuracy: 0.866
